In [ ]:
%matplotlib inline

# Sobol' graph for the Sobieski's SSBJ use case.


In [ ]:
from __future__ import annotations

from gemseo.algos.design_space import DesignSpace
from gemseo.problems.mdo.sobieski.core.problem import SobieskiProblem
from gemseo.problems.mdo.sobieski.disciplines import SobieskiAerodynamics
from gemseo.problems.mdo.sobieski.disciplines import SobieskiMission
from gemseo.problems.mdo.sobieski.disciplines import SobieskiPropulsion
from gemseo.problems.mdo.sobieski.disciplines import SobieskiStructure
from gemseo.uncertainty.sensitivity.sobol_analysis import SobolAnalysis
from gemseo.utils.data_conversion import split_array_to_dict_of_arrays

from gemseo_umdo.visualizations.sobol_graph import SobolGraph

First,
we define an uncertain space around the optimum design:



In [ ]:
design_space = SobieskiProblem().design_space
design_variable_names = ["x_1", "x_2", "x_3", "x_shared"]
design_space.filter(design_variable_names)
optimum_design = split_array_to_dict_of_arrays(
    SobieskiProblem().optimum_design,
    design_space.variable_sizes,
    design_variable_names,
)

uncertain_space = DesignSpace()
for name, value in optimum_design.items():
    uncertain_space.add_variable(
        name,
        size=value.size,
        lower_bound=value * 0.95,
        upper_bound=value * 1.05,
        value=value,
    )

Then,
we define the disciplines:



In [ ]:
disciplines = [
    SobieskiAerodynamics(),
    SobieskiStructure(),
    SobieskiPropulsion(),
    SobieskiMission(),
]

Thirdly,
we compute the Sobol' indices for all the outputs of the MDO problem:



In [ ]:
sobol_analysis = SobolAnalysis()
sobol_analysis.compute_samples(disciplines, uncertain_space, 100)
sobol_analysis.compute_indices()

Lastly,
we draw the Sobol' graph:



In [ ]:
sobol_graph = SobolGraph.from_analysis(sobol_analysis, output_name="y_4")
sobol_graph